In [1]:
import pandas as pd
import numpy as np

anime = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation_systems/anime_data/anime.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation_systems/anime_data/rating.csv/rating.csv')
anime['genre'] = anime['genre'].astype(str)
anime['main genre'] = anime['genre'].str.split(', ').str[0]
## can use main genre to look at specific genres

# need to remove -1s and change the unknowns

In [45]:
anime = anime[anime['rating'] != -1]
ratings = ratings[ratings['rating'] != -1]

anime = anime.drop(list(anime.loc[anime['rating'].isnull() == True].index), axis=0) ##.index and axis = 0  to ensure it only works on the one column
anime = anime.fillna('Unknown') ## replaces any blanks in non-numeric data
anime =  anime.reset_index(drop = True)

## want to rec based on those with highest ratings and community members

In [46]:
rating_count = anime.groupby('name')['rating'].count().sort_values(ascending = False)
print(f"number of ratings: {rating_count}")

rating_sum = anime.groupby('name')['rating'].sum().sort_values(ascending = False)
print(f"sum of ratings: {rating_sum}\n")

rating_mean = anime.groupby('name')['rating'].mean().sort_values(ascending = False)
print(f"average rating : {rating_mean}\n")

number of ratings: name
Shi Wan Ge Leng Xiaohua                         2
Saru Kani Gassen                                2
Otome Hime                                      1
Otome Juurin Yuugi: Maiden Infringement Play    1
Otome Nadeshiko Koi Techou                      1
                                               ..
Himitsukessha Taka no Tsume                     1
Himitsukessha Taka no Tsume Countdown           1
Himitsukessha Taka no Tsume DO                  1
Himitsukessha Taka no Tsume EX                  1
Himiko-den                                      1
Name: rating, Length: 12062, dtype: int64
sum of ratings: name
Shi Wan Ge Leng Xiaohua                    13.74
Taka no Tsume 8: Yoshida-kun no X-Files    10.00
Saru Kani Gassen                            9.98
Spoon-hime no Swing Kitchen                 9.60
Mogura no Motoro                            9.50
                                           ...  
Hametsu no Mars                             2.37
Utsu Musume Sayuri  

In [47]:
members_count = anime.groupby('name')['members'].count().sort_values(ascending = False)
print(f"number of memberss: {members_count}")

members_sum = anime.groupby('name')['members'].sum().sort_values(ascending = False)
print(f"sum of memberss: {members_sum}\n")

members_mean = anime.groupby('name')['members'].mean().sort_values(ascending = False)
print(f"average members : {members_mean}\n")

number of memberss: name
Shi Wan Ge Leng Xiaohua                         2
Saru Kani Gassen                                2
Otome Hime                                      1
Otome Juurin Yuugi: Maiden Infringement Play    1
Otome Nadeshiko Koi Techou                      1
                                               ..
Himitsukessha Taka no Tsume                     1
Himitsukessha Taka no Tsume Countdown           1
Himitsukessha Taka no Tsume DO                  1
Himitsukessha Taka no Tsume EX                  1
Himiko-den                                      1
Name: members, Length: 12062, dtype: int64
sum of memberss: name
Death Note                                 1013917
Shingeki no Kyojin                          896229
Sword Art Online                            893100
Fullmetal Alchemist: Brotherhood            793665
Angel Beats!                                717796
                                            ...   
Chotto Ugoku!? &quot;Futeneko&quot;             17
War

In [48]:
print(f"rating info: {rating_mean.describe()}")
print(f"members info: {members_mean.describe()}")


rating info: count    12062.000000
mean         6.473992
std          1.026728
min          1.670000
25%          5.880000
50%          6.570000
75%          7.180000
max         10.000000
Name: rating, dtype: float64
members info: count    1.206200e+04
mean     1.828254e+04
std      5.527987e+04
min      1.200000e+01
25%      2.212500e+02
50%      1.539000e+03
75%      9.486500e+03
max      1.013917e+06
Name: members, dtype: float64


## Decide what is popular

In [52]:
## both chosen for being around the 75% mean mark

popular_rated = 7
popular_members = 10000

In [53]:
pop_rated_animes = rating_mean[rating_mean >= popular_rated]
pop_anime_communities = members_mean[members_mean >= popular_members]

In [54]:
pop_rated_animes.sort_values(ascending = False).head(10)

,rating
name,
Taka no Tsume 8: Yoshida-kun no X-Files,10.00
Spoon-hime no Swing Kitchen,9.60
Mogura no Motoro,9.50
Kimi no Na wa.,9.37
Kahei no Umi,9.33
Fullmetal Alchemist: Brotherhood,9.26
Yakusoku: Africa Mizu to Midori,9.25
Gintama°,9.25
Steins;Gate,9.17


In [55]:
pop_anime_communities.sort_values(ascending = False).head(10)

,members
name,
Death Note,1013917.0
Shingeki no Kyojin,896229.0
Sword Art Online,893100.0
Fullmetal Alchemist: Brotherhood,793665.0
Angel Beats!,717796.0
Code Geass: Hangyaku no Lelouch,715151.0
Naruto,683297.0
Steins;Gate,673572.0
Mirai Nikki (TV),657190.0


## popular per genre
-- using main genre

In [56]:
for genre in pop_rated_animes('main genre').unique():
    print(f"top anime in {genre}: {pop_rated_animes.sort_values(ascending = False).head(1)}")

TypeError: 'Series' object is not callable

In [57]:
for genre in anime[anime['name'].isin(pop_rated_animes.index)]['main genre'].unique():
    print(f"top anime in {genre}: {pop_rated_animes[pop_rated_animes.index.isin(anime[anime['main genre'] == genre]['name'])].sort_values(ascending = False).head(1)}")

top anime in ['Drama': name
Kimi no Na wa.    9.37
Name: rating, dtype: float64
top anime in ['Action': name
Fullmetal Alchemist: Brotherhood    9.26
Name: rating, dtype: float64
top anime in ['Sci-Fi': name
Steins;Gate    9.17
Name: rating, dtype: float64
top anime in ['Comedy': name
Taka no Tsume 8: Yoshida-kun no X-Files    10.0
Name: rating, dtype: float64
top anime in ['Adventure': name
Spoon-hime no Swing Kitchen    9.6
Name: rating, dtype: float64
top anime in ['Fantasy': name
Santa-san wa Dai Isogashi    9.0
Name: rating, dtype: float64
top anime in ['Mystery': name
Death Note    8.71
Name: rating, dtype: float64
top anime in ['Psychological': name
Ping Pong The Animation    8.67
Name: rating, dtype: float64
top anime in ['Ecchi': name
Shokugeki no Souma    8.61
Name: rating, dtype: float64
top anime in ['Josei': name
Usagi Drop    8.56
Name: rating, dtype: float64
top anime in ['Military': name
Girls und Panzer der Film    8.55
Name: rating, dtype: float64
top anime in ['Roman